In [1]:
from typing import Any, OrderedDict, List, Dict
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.models.resnet import resnet50
from torchvision.transforms import v2 as T
import matplotlib.pyplot as plt
import json
from torchmetrics import Accuracy
from lightly.transforms.utils import IMAGENET_NORMALIZE
from PIL import Image
from camelyon_ds import Camelyon17Dataset

## Load Data

In [2]:
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(
        mean=IMAGENET_NORMALIZE["mean"],
        std=IMAGENET_NORMALIZE["std"],
    ),
])

/home/yasin/miniforge3/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [3]:
train_set = Camelyon17Dataset('/data', train=True, transform=transform)
test_set = Camelyon17Dataset('/data', train=False, transform=transform)

len(train_set), len(test_set)

(22000, 2000)

In [4]:
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=4, persistent_workers=True, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=256, shuffle=False, num_workers=4, persistent_workers=True, pin_memory=True)

## Train Models

In [5]:
import pytorch_lightning as L

In [6]:
from typing import Any


class LinearProbe(L.LightningModule):
    def __init__(self, backbone, emb_dim, num_classes, lr=1e-2) -> None:
        super().__init__()
        
        self.backbone: nn.Module = backbone
        for param in self.backbone.parameters():
            param.requires_grad = False

        self.linear_head: nn.Module = nn.Linear(emb_dim, num_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.lr = lr

        self.test_accuracy = Accuracy(task='multiclass', num_classes=num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.linear_head(x)

        return x

    def training_step(self, batch, bacth_idx):
        X = batch['image']
        t = batch['label']

        y = self.forward(X)
        loss = self.criterion(y, t)

        print(loss.item())

        return loss
    
    def test_step(self, batch, batch_idx):
        X = batch['image']
        t = batch['label']

        y = self.forward(X)
        acc = self.test_accuracy(y, t)

        self.log('accuracy', acc, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.linear_head.parameters(), lr=self.lr)

In [7]:
def get_backbone_from_ckpt(ckpt_path: str) -> torch.nn.Module:
    state_dict = torch.load(ckpt_path)["state_dict"]
    state_dict = OrderedDict([
        (".".join(name.split(".")[1:]), param) for name, param in state_dict.items() if name.startswith("backbone")
    ])

    return state_dict

In [8]:
# Baseline Model
model_bl  = resnet50()
weights_bl = get_backbone_from_ckpt("./data/r50_camelyon_bl.ckpt")
# model_bl.load_state_dict(weights_bl, strict=False)
model_bl.fc = torch.nn.Identity()
model_bl = model_bl.cuda()

In [9]:
# DANN Model
model_ms  = resnet50()
weights_ms = get_backbone_from_ckpt("./data/r50_camelyon_ms.ckpt")
model_ms.load_state_dict(weights_ms, strict=False)
model_ms.fc = torch.nn.Identity()
model_ms = model_ms.cuda()

In [10]:
baseline_module = LinearProbe(model_bl, emb_dim=2048, num_classes=2, lr=3e-3)

In [11]:
mixstyle_module = LinearProbe(model_ms, emb_dim=2048, num_classes=2, lr=3e-3)

In [12]:
trainer = L.Trainer(
    max_epochs=20
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(baseline_module, train_loader)

Missing logger folder: /home/yasin/repos/dispatch_smol/notebooks/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | backbone      | ResNet             | 23.5 M
1 | linear_head   | Linear             | 4.1 K 
2 | criterion     | CrossEntropyLoss   | 0     
3 | test_accuracy | MulticlassAccuracy | 0     
-----------------------------------------------------
4.1 K     Trainable params
23.5 M    Non-trainable params
23.5 M    Total params
94.049    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

0.7572866082191467
3.9290578365325928
2.774915933609009
0.6622446179389954
2.061999559402466
2.0032565593719482
0.7148746252059937
0.42608240246772766
1.213427186012268
0.9795317053794861
0.7281064987182617
0.3899158835411072
0.38213348388671875
0.6596354842185974
0.8489046692848206
0.483634352684021
0.5864026546478271
0.33289241790771484
0.45170801877975464
0.4818207323551178
0.6171343922615051
0.4034325182437897
0.4663030207157135
0.26251381635665894
0.4592558741569519
0.4399016499519348
0.5978531241416931
0.5522154569625854
0.45452702045440674
0.3972989618778229
0.6137452721595764
0.2941893935203552
0.4184429943561554
0.5407396554946899
0.3065659701824188
0.5505945086479187
0.45381295680999756
0.6851460337638855
0.4334942400455475
0.44244712591171265
0.45116662979125977
0.44158127903938293
0.47751522064208984
0.3183057904243469
0.431462824344635
0.36991727352142334
0.421236127614975
0.44956904649734497
0.5176919102668762
0.41481080651283264
0.39406606554985046
0.4101826846599579
0.3

`Trainer.fit` stopped: `max_epochs=20` reached.


In [14]:
trainer.test(baseline_module, dataloaders=train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9165909290313721     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.9165909290313721}]

In [15]:
trainer.test(baseline_module, dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.7275000214576721     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.7275000214576721}]

In [16]:
trainer = L.Trainer(
    max_epochs=20
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
trainer.fit(mixstyle_module, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | backbone      | ResNet             | 23.5 M
1 | linear_head   | Linear             | 4.1 K 
2 | criterion     | CrossEntropyLoss   | 0     
3 | test_accuracy | MulticlassAccuracy | 0     
-----------------------------------------------------
4.1 K     Trainable params
23.5 M    Non-trainable params
23.5 M    Total params
94.049    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

0.7286106944084167
0.6350235939025879
0.31535670161247253
0.3379063010215759
0.16024762392044067
0.23151257634162903
0.23007337749004364
0.1791251003742218
0.2218264639377594
0.0631585419178009
0.07308293879032135
0.12022079527378082
0.07320050150156021
0.11332400143146515
0.1938868761062622
0.07947340607643127
0.12617896497249603
0.16718682646751404
0.14333230257034302
0.09338295459747314
0.07457834482192993
0.03994220867753029
0.03555695340037346
0.14209558069705963
0.06898410618305206
0.05978809669613838
0.055516310036182404
0.0689685121178627
0.050158388912677765
0.06647327542304993
0.08152786642313004
0.06967445462942123
0.0397907979786396
0.05692503601312637
0.07187967747449875
0.06507979333400726
0.1144070029258728
0.14073550701141357
0.05981466546654701
0.0854640007019043
0.06024620309472084
0.05029035732150078
0.06458472460508347
0.0893993079662323
0.07247985899448395
0.05639662593603134
0.05690547823905945
0.06046249717473984
0.05562244728207588
0.08504980057477951
0.05473986

`Trainer.fit` stopped: `max_epochs=20` reached.


In [18]:
trainer.test(mixstyle_module, dataloaders=train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │     0.995090901851654     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.995090901851654}]

In [19]:
trainer.test(mixstyle_module, dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9600000381469727     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.9600000381469727}]